<center style='font-size:40px'><b>Feature Engineering</b></center>

# Data Overview:
## Import Libraries:

In [1]:
import os
import pickle
import warnings
import numpy as np
from util import *
%matplotlib inline
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
pd.options.display.max_columns = 1000
from tqdm import tqdm_notebook as tqdm

## Read Data:

In [2]:
train = pd.read_csv('../2_Data Preprocessing/output/train_processed.csv')
test  = pd.read_csv('../2_Data Preprocessing/output/test_processed.csv')


shape(train, test)

~> [train] has  1,460  rows, and  77  columns.
~> [test ] has  1,459  rows, and  76  columns.


In [3]:
# Combine train and test together to apply the changes.
traintest     = pd.concat([train, test], axis=0)
train_ids     = train.Id
train_labels  = train.SalePrice
test_ids      = test.Id

shape(traintest)

~> [traintest] has  2,919  rows, and  77  columns.


# Feature Generation: 
## Categorical Features:

In [4]:
# Some utility function:
def save_or_load(name):
    # See if list colums is there or not.
    if os.path.isfile(f'./{name}.pkl'):
        with open(f'{name}.pkl', 'rb') as f: 
            list_cols = pickle.load(f)
    else:
        # Select only the list colums.
        list_cols = [col for col in train.columns if col.startswith(name)]

        # Save the list colums to later use.
        with open(f'{name}.pkl', 'wb') as f:
            pickle.dump(list_cols, f)
    return list_cols

In [5]:
ord_cols  = ['LandSlope', 'OverallQual', 'OverallCond', 'ExterQual', 'ExterCond', 'BsmtExposure', 
             'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'BsmtFinType2', 'HeatingQC', 'Electrical', 
             'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
             'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond', 'PavedDrive',
             'PoolQC', 'BedroomAbvGr', 'KitchenAbvGr']
cat_cols  = ['MSSubClass', 'MSZoning', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 
            'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'RoofStyle', 'RoofMatl', 'Exterior1st',
            'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating', 'GarageType', 'MiscFeature', 'SaleType', 
             'SaleCondition', 'HouseStyle', 'Fence', 'CentralAir']
num_cols  = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
            '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
            'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
date_cols = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']

### One-Hot Encoding:

In [6]:
for var in list(ord_cols+cat_cols):
    train = pd.concat([train, pd.get_dummies(train[var], prefix=f'onehot_{var}', prefix_sep='_', drop_first=True)], axis=1)
    test  = pd.concat([test, pd.get_dummies(test[var], prefix=f'onehot_{var}', prefix_sep='_', drop_first=True)], axis=1)

shape(train, test)

~> [train] has  1,460  rows, and  367  columns.
~> [test ] has  1,459  rows, and  351  columns.


In [7]:
# Align all the columns in train data with test data.
train_labels       = train.SalePrice
train, test        = train.align(test, join='inner', axis=1)
train['SalePrice'] = train_labels

shape(train, test)

~> [train] has  1,460  rows, and  337  columns.
~> [test ] has  1,459  rows, and  336  columns.


We can see after aligning the train and test datasets, there are number of columns dropped.

In [8]:
onehot_cols = save_or_load('onehot_cols')

### Frequency Encoding:

In [9]:
for col in list(cat_cols+ord_cols):
    encoding             = traintest[col].value_counts(normalize=True)
    train[f'freq_{col}'] = train[col].apply(lambda x: encoding[x])
    test[f'freq_{col}']  = test[col].apply(lambda x: encoding[x])

shape(train, test)

~> [train] has  1,460  rows, and  390  columns.
~> [test ] has  1,459  rows, and  389  columns.


In [10]:
freq_cols = save_or_load('freq_cols')

### Target Encoding:

In [11]:
for col in list(cat_cols+ord_cols):
    means   = train.groupby(col).SalePrice.mean()
    stds    = train.groupby(col).SalePrice.std()
    skews   = train.groupby(col).SalePrice.skew()
    counts  = train.groupby(col).SalePrice.count()
    medians = train.groupby(col).SalePrice.median()
    mins    = train.groupby(col).SalePrice.min()
    maxs    = train.groupby(col).SalePrice.max()
    
    for stat in [means, stds, skews, medians, mins, maxs]:
        train[f'target_enc_{var2str(stat)}_{col}'] = train[col].map(stat)
        test[f'target_enc_{var2str(stat)}_{col}']  = test[col].map(stat)

In [12]:
# save or load the target encoded columns' names.
target_enc_cols = save_or_load('target_enc_cols')

In [13]:
shape(train, test)

~> [train] has  1,460  rows, and  708  columns.
~> [test ] has  1,459  rows, and  707  columns.


<div class='alert alert-info'>
    <p style='font-size:20px;font-weight:bold'>Note:<p>
    <p style='font-size:16px'>We can see that the number of columns increases as we go along. <br>In the next notebook, we'll apply some feature selection techniques to get rid of the redundant features.</p>
</div>

## Numerical Features:
### Log1p / Sqrt:

In [14]:
for col in num_cols:
    train[f'log_{col}'] = np.log1p(train[col])
    train[f'sqrt_{col}'] = np.sqrt(train[col])
    
    test[f'log_{col}'] = np.log1p(test[col])
    test[f'sqrt_{col}'] = np.sqrt(test[col])

# See the shape of both datsets.
shape(train, test)

~> [train] has  1,460  rows, and  742  columns.
~> [test ] has  1,459  rows, and  741  columns.


**Save the name of the columns for later use**

In [15]:
# save or load the log/sqrt columns' names.
log_cols = save_or_load('log_cols')
sqrt_cols = save_or_load('sqrt_cols')

### Polynomial Features:

In [20]:
from sklearn.preprocessing import PolynomialFeatures

poly_transformer = PolynomialFeatures(degree=3, include_bias=False)
poly_transformer.fit(train[num_cols])

poly_train = poly_transformer.transform(train[num_cols])
poly_test  = poly_transformer.transform(test[num_cols])

shape(poly_train, poly_test)

~> [poly_train] has  1,460  rows, and  1,139  columns.
~> [poly_test ] has  1,459  rows, and  1,139  columns.


In [21]:
# Convert it to Dataframe
poly_train = pd.DataFrame(poly_train, 
                          columns=poly_transformer.get_feature_names(num_cols)).drop(num_cols, axis=1)
poly_test  = pd.DataFrame(poly_test,
                          columns=poly_transformer.get_feature_names(num_cols)).drop(num_cols, axis=1)
poly_train['SalePrice'] = train.SalePrice

In [22]:
poly_train.head()

,LotFrontage^2,LotFrontage LotArea,LotFrontage MasVnrArea,LotFrontage BsmtFinSF1,LotFrontage BsmtFinSF2,LotFrontage BsmtUnfSF,LotFrontage TotalBsmtSF,LotFrontage 1stFlrSF,LotFrontage 2ndFlrSF,LotFrontage LowQualFinSF,LotFrontage GrLivArea,LotFrontage GarageArea,LotFrontage WoodDeckSF,LotFrontage OpenPorchSF,LotFrontage EnclosedPorch,LotFrontage 3SsnPorch,LotFrontage ScreenPorch,LotArea^2,LotArea MasVnrArea,LotArea BsmtFinSF1,LotArea BsmtFinSF2,LotArea BsmtUnfSF,LotArea TotalBsmtSF,LotArea 1stFlrSF,LotArea 2ndFlrSF,LotArea LowQualFinSF,LotArea GrLivArea,LotArea GarageArea,LotArea WoodDeckSF,LotArea OpenPorchSF,LotArea EnclosedPorch,LotArea 3SsnPorch,LotArea ScreenPorch,MasVnrArea^2,MasVnrArea BsmtFinSF1,MasVnrArea BsmtFinSF2,MasVnrArea BsmtUnfSF,MasVnrArea TotalBsmtSF,MasVnrArea 1stFlrSF,MasVnrArea 2ndFlrSF,MasVnrArea LowQualFinSF,MasVnrArea GrLivArea,MasVnrArea GarageArea,MasVnrArea WoodDeckSF,MasVnrArea OpenPorchSF,MasVnrArea EnclosedPorch,MasVnrArea 3SsnPorch,MasVnrArea ScreenPorch,BsmtFinSF1^2,BsmtFinSF1 BsmtFinSF2,BsmtFinSF1 BsmtUnfSF,BsmtFinSF1 TotalBsmtSF,BsmtFinSF1 1stFlrSF,BsmtFinSF1 2ndFlrSF,BsmtFinSF1 LowQualFinSF,BsmtFinSF1 GrLivArea,BsmtFinSF1 GarageArea,BsmtFinSF1 WoodDeckSF,BsmtFinSF1 OpenPorchSF,BsmtFinSF1 EnclosedPorch,BsmtFinSF1 3SsnPorch,BsmtFinSF1 ScreenPorch,BsmtFinSF2^2,BsmtFinSF2 BsmtUnfSF,BsmtFinSF2 TotalBsmtSF,BsmtFinSF2 1stFlrSF,BsmtFinSF2 2ndFlrSF,BsmtFinSF2 LowQualFinSF,BsmtFinSF2 GrLivArea,BsmtFinSF2 GarageArea,BsmtFinSF2 WoodDeckSF,BsmtFinSF2 OpenPorchSF,BsmtFinSF2 EnclosedPorch,BsmtFinSF2 3SsnPorch,BsmtFinSF2 ScreenPorch,BsmtUnfSF^2,BsmtUnfSF TotalBsmtSF,BsmtUnfSF 1stFlrSF,BsmtUnfSF 2ndFlrSF,BsmtUnfSF LowQualFinSF,BsmtUnfSF GrLivArea,BsmtUnfSF GarageArea,BsmtUnfSF WoodDeckSF,BsmtUnfSF OpenPorchSF,BsmtUnfSF EnclosedPorch,BsmtUnfSF 3SsnPorch,BsmtUnfSF ScreenPorch,TotalBsmtSF^2,TotalBsmtSF 1stFlrSF,TotalBsmtSF 2ndFlrSF,TotalBsmtSF LowQualFinSF,TotalBsmtSF GrLivArea,TotalBsmtSF GarageArea,TotalBsmtSF WoodDeckSF,TotalBsmtSF OpenPorchSF,TotalBsmtSF EnclosedPorch,TotalBsmtSF 3SsnPorch,TotalBsmtSF ScreenPorch,1stFlrSF^2,1stFlrSF 2ndFlrSF,1stFlrSF LowQualFinSF,1stFlrSF GrLivArea,1stFlrSF GarageArea,1stFlrSF WoodDeckSF,1stFlrSF OpenPorchSF,1stFlrSF EnclosedPorch,1stFlrSF 3SsnPorch,1stFlrSF ScreenPorch,2ndFlrSF^2,2ndFlrSF LowQualFinSF,2ndFlrSF GrLivArea,2ndFlrSF GarageArea,2ndFlrSF WoodDeckSF,2ndFlrSF OpenPorchSF,2ndFlrSF EnclosedPorch,2ndFlrSF 3SsnPorch,2ndFlrSF ScreenPorch,LowQualFinSF^2,LowQualFinSF GrLivArea,LowQualFinSF GarageArea,LowQualFinSF WoodDeckSF,LowQualFinSF OpenPorchSF,LowQualFinSF EnclosedPorch,LowQualFinSF 3SsnPorch,LowQualFinSF ScreenPorch,GrLivArea^2,GrLivArea GarageArea,GrLivArea WoodDeckSF,GrLivArea OpenPorchSF,GrLivArea EnclosedPorch,GrLivArea 3SsnPorch,GrLivArea ScreenPorch,GarageArea^2,GarageArea WoodDeckSF,GarageArea OpenPorchSF,GarageArea EnclosedPorch,GarageArea 3SsnPorch,GarageArea ScreenPorch,WoodDeckSF^2,WoodDeckSF OpenPorchSF,WoodDeckSF EnclosedPorch,WoodDeckSF 3SsnPorch,WoodDeckSF ScreenPorch,OpenPorchSF^2,OpenPorchSF EnclosedPorch,OpenPorchSF 3SsnPorch,OpenPorchSF ScreenPorch,EnclosedPorch^2,EnclosedPorch 3SsnPorch,EnclosedPorch ScreenPorch,3SsnPorch^2,3SsnPorch ScreenPorch,ScreenPorch^2,LotFrontage^3,LotFrontage^2 LotArea,LotFrontage^2 MasVnrArea,LotFrontage^2 BsmtFinSF1,LotFrontage^2 BsmtFinSF2,LotFrontage^2 BsmtUnfSF,LotFrontage^2 TotalBsmtSF,LotFrontage^2 1stFlrSF,LotFrontage^2 2ndFlrSF,LotFrontage^2 LowQualFinSF,LotFrontage^2 GrLivArea,LotFrontage^2 GarageArea,LotFrontage^2 WoodDeckSF,LotFrontage^2 OpenPorchSF,LotFrontage^2 EnclosedPorch,LotFrontage^2 3SsnPorch,LotFrontage^2 ScreenPorch,LotFrontage LotArea^2,LotFrontage LotArea MasVnrArea,LotFrontage LotArea BsmtFinSF1,LotFrontage LotArea BsmtFinSF2,LotFrontage LotArea BsmtUnfSF,LotFrontage LotArea TotalBsmtSF,LotFrontage LotArea 1stFlrSF,LotFrontage LotArea 2ndFlrSF,LotFrontage LotArea LowQualFinSF,LotFrontage LotArea GrLivArea,LotFrontage LotArea GarageArea,LotFrontage LotArea WoodDeckSF,LotFrontage LotArea OpenPorchSF

<div class='alert alert-info'>
    <p style='font-size:20px;font-weight:bold'>Note:<p>
    <p style='font-size:16px'>- As we can see the polynomial features exceeds the number of column, which turn us into the problem that is called <b>Curse of Dimensionality</b>, so to reduce the effect of that curse, we need to reduce it.<br><br>
     - As we have a huge number of columns, there is a high probability that we are under multi-collinearity, and I'm going to see that by computing the correlation between columns, and remove the ones that have high correlation.
    </p>
    
</div>

In [26]:
# Create correlation matrix
corr_matrix = poly_train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find index of feature columns with correlation greater than 0.95
to_drop = [col for col in upper.columns if any(upper[col] > 0.95)]

In [28]:
poly_train[to_drop].head()

,LotArea 1stFlrSF,LotArea GrLivArea,MasVnrArea 1stFlrSF,BsmtFinSF1^2,BsmtFinSF1 TotalBsmtSF,BsmtFinSF1 1stFlrSF,BsmtFinSF1 GrLivArea,BsmtFinSF2 1stFlrSF,BsmtFinSF2 GrLivArea,BsmtUnfSF 1stFlrSF,TotalBsmtSF 1stFlrSF,TotalBsmtSF 3SsnPorch,1stFlrSF 2ndFlrSF,1stFlrSF LowQualFinSF,1stFlrSF GarageArea,1stFlrSF WoodDeckSF,1stFlrSF OpenPorchSF,1stFlrSF EnclosedPorch,1stFlrSF 3SsnPorch,1stFlrSF ScreenPorch,2ndFlrSF GrLivArea,LowQualFinSF^2,LowQualFinSF GrLivArea,GrLivArea 3SsnPorch,GarageArea 3SsnPorch,GarageArea ScreenPorch,LotFrontage^2 BsmtFinSF2,LotFrontage^2 TotalBsmtSF,LotFrontage^2 GrLivArea,LotFrontage^2 GarageArea,LotFrontage^2 3SsnPorch,LotFrontage^2 ScreenPorch,LotFrontage LotArea BsmtFinSF2,LotFrontage LotArea TotalBsmtSF,LotFrontage LotArea 1stFlrSF,LotFrontage LotArea LowQualFinSF,LotFrontage LotArea GrLivArea,LotFrontage LotArea GarageArea,LotFrontage LotArea OpenPorchSF,LotFrontage LotArea EnclosedPorch,LotFrontage LotArea 3SsnPorch,LotFrontage LotArea ScreenPorch,LotFrontage MasVnrArea BsmtFinSF2,LotFrontage MasVnrArea BsmtUnfSF,LotFrontage MasVnrArea TotalBsmtSF,LotFrontage MasVnrArea 1stFlrSF,LotFrontage MasVnrArea GrLivArea,LotFrontage MasVnrArea GarageArea,LotFrontage MasVnrArea 3SsnPorch,LotFrontage MasVnrArea ScreenPorch,LotFrontage BsmtFinSF1^2,LotFrontage BsmtFinSF1 TotalBsmtSF,LotFrontage BsmtFinSF1 1stFlrSF,LotFrontage BsmtFinSF1 GrLivArea,LotFrontage BsmtFinSF1 GarageArea,LotFrontage BsmtFinSF1 OpenPorchSF,LotFrontage BsmtFinSF1 3SsnPorch,LotFrontage BsmtFinSF1 ScreenPorch,LotFrontage BsmtFinSF2^2,LotFrontage BsmtFinSF2 BsmtUnfSF,LotFrontage BsmtFinSF2 TotalBsmtSF,LotFrontage BsmtFinSF2 1stFlrSF,LotFrontage BsmtFinSF2 2ndFlrSF,LotFrontage BsmtFinSF2 GrLivArea,LotFrontage BsmtFinSF2 GarageArea,LotFrontage BsmtFinSF2 WoodDeckSF,LotFrontage BsmtFinSF2 OpenPorchSF,LotFrontage BsmtFinSF2 EnclosedPorch,LotFrontage BsmtFinSF2 3SsnPorch,LotFrontage BsmtFinSF2 ScreenPorch,LotFrontage BsmtUnfSF^2,LotFrontage BsmtUnfSF 1stFlrSF,LotFrontage BsmtUnfSF WoodDeckSF,LotFrontage BsmtUnfSF EnclosedPorch,LotFrontage BsmtUnfSF 3SsnPorch,LotFrontage BsmtUnfSF ScreenPorch,LotFrontage TotalBsmtSF^2,LotFrontage TotalBsmtSF 1stFlrSF,LotFrontage TotalBsmtSF LowQualFinSF,LotFrontage TotalBsmtSF GrLivArea,LotFrontage TotalBsmtSF GarageArea,LotFrontage TotalBsmtSF OpenPorchSF,LotFrontage TotalBsmtSF 3SsnPorch,LotFrontage TotalBsmtSF ScreenPorch,LotFrontage 1stFlrSF^2,LotFrontage 1stFlrSF 2ndFlrSF,LotFrontage 1stFlrSF LowQualFinSF,LotFrontage 1stFlrSF GrLivArea,LotFrontage 1stFlrSF GarageArea,LotFrontage 1stFlrSF WoodDeckSF,LotFrontage 1stFlrSF OpenPorchSF,LotFrontage 1stFlrSF EnclosedPorch,LotFrontage 1stFlrSF 3SsnPorch,LotFrontage 1stFlrSF ScreenPorch,LotFrontage 2ndFlrSF GrLivArea,LotFrontage 2ndFlrSF 3SsnPorch,LotFrontage 2ndFlrSF ScreenPorch,LotFrontage LowQualFinSF^2,LotFrontage LowQualFinSF GrLivArea,LotFrontage LowQualFinSF WoodDeckSF,LotFrontage LowQualFinSF OpenPorchSF,LotFrontage LowQualFinSF EnclosedPorch,LotFrontage LowQualFinSF 3SsnPorch,LotFrontage LowQualFinSF ScreenPorch,LotFrontage GrLivArea^2,LotFrontage GrLivArea GarageArea,LotFrontage GrLivArea WoodDeckSF,LotFrontage GrLivArea OpenPorchSF,LotFrontage GrLivArea EnclosedPorch,LotFrontage GrLivArea 3SsnPorch,LotFrontage GrLivArea ScreenPorch,LotFrontage GarageArea OpenPorchSF,LotFrontage GarageArea 3SsnPorch,LotFrontage GarageArea ScreenPorch,LotFrontage WoodDeckSF^2,LotFrontage WoodDeckSF EnclosedPorch,LotFrontage WoodDeckSF 3SsnPorch,LotFrontage WoodDeckSF ScreenPorch,LotFrontage OpenPorchSF 3SsnPorch,LotFrontage OpenPorchSF ScreenPorch,LotFrontage EnclosedPorch^2,LotFrontage EnclosedPorch 3SsnPorch,LotFrontage EnclosedPorch ScreenPorch,LotFrontage 3SsnPorch^2,LotFrontage ScreenPorch^2,LotArea^3,LotArea^2 BsmtFinSF1,LotArea^2 BsmtFinSF2,LotArea^2 TotalBsmtSF,LotArea^2 1stFlrSF,LotArea^2 LowQualFinSF,LotArea^2 GrLivArea,LotArea^2 GarageArea,LotArea^2 3SsnPorch,LotArea MasVnrArea BsmtFinSF1,LotArea MasVnrArea TotalBsmtSF,LotArea MasVnrArea 1stFlrSF,LotArea MasVnrArea GrLi

In [30]:
# Remove the higly correlated features.
poly_train.drop(to_drop+['SalePrice'], axis=1, inplace=True)
poly_test.drop(to_drop, axis=1, inplace=True)

In [31]:
shape(poly_train, poly_test)

~> [poly_train] has  1,460  rows, and  480  columns.
~> [poly_test ] has  1,459  rows, and  480  columns.


In [32]:
# Since it takes a quite huge time, I'll save it for later use.
poly_train = reduce_

poly_train.to_csv('./output/poly_train.csv', index=False)
poly_test.to_csv('./output/poly_test.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: './output/poly_train.csv'